####Data Merging and Exploratory Analysis
######Data Mining II Final Project
######Eugene Lucino


Merging of datasets to create a dataset useful for linear regression and acquiring some additional statistics.
Some of the code in this file is based on Andy C. Dick's Feature Engineering code he used when making his predictions for the Tokyo Olympics. The major differences from his code are in the variables used to make a model.

In [ ]:
import pandas as pd
import io

In [ ]:
# import "medals" and "nationslist" csv files
from google.colab import drive
from google.colab import files
uploaded = files.upload()

Saving medals.csv to medals.csv
Saving nationslist.csv to nationslist.csv


In [ ]:
# read csv files
medals = pd.read_csv(io.BytesIO(uploaded['medals.csv']))
economy = pd.read_csv(io.BytesIO(uploaded['nationslist.csv']), encoding='latin-1')

###### Merging Files

In [ ]:
# merge the two files based on NOC code
games = medals.merge(economy, on='NOC', sort=True)
games = games[["Year","Country","Population","GDP_Per_Capita","Host_NOC","Host_City","Sport","Event","NOC","Medal"]]
games

,Year,Country,Population,GDP_Per_Capita,Host_NOC,Host_City,Sport,Event,NOC,Medal
0,2008,Afghanistan,37172386,520.90,CHN,Beijing,Taekwondo,Taekwondo Men's Flyweight,AFG,Bronze
1,2012,Afghanistan,37172386,520.90,GBR,London,Taekwondo,Taekwondo Men's Featherweight,AFG,Bronze
2,2000,Algeria,42228429,4114.72,AUS,Sydney,Boxing,Boxing Men's Light-Welterweight,ALG,Bronze
3,1996,Algeria,42228429,4114.72,USA,Atlanta,Boxing,Boxing Men's Middleweight,ALG,Bronze
4,2008,Algeria,42228429,4114.72,CHN,Beijing,Judo,Judo Men's Middleweight,ALG,Silver
...,...,...,...,...,...,...,...,...,...,...
6267,2004,Zimbabwe,14439018,2147.00,GRE,Athina,Swimming,Swimming Women's 200 metres Individual Medley,ZIM,Bronze
6268,2008,Zimbabwe,14439018,2147.00,CHN,Beijing,Swimming,Swimming Women's 100 metres Backstroke,ZIM,Silver
6269,2008,Zimbabwe,14439018,2147.00,CHN,Beijing,Swimming,Swimming Women's 200 metres Backstroke,ZIM,Gold
6270,2008,Zimbabwe,14439018,2147.00,CHN,Beijing,Swimming,Swimming Women's 200 metres Individual Medley,ZIM,Silver


In [ ]:
# creating new variables based on which type of medal was won
# and if the country was the host nation at the time
games['Gold']   = games['Medal'] == 'Gold'
games['Silver'] = games['Medal'] == 'Silver'
games['Bronze'] = games['Medal'] == 'Bronze'
games['Home']   = games['NOC']   == games['Host_NOC']
games

,Year,Country,Population,GDP_Per_Capita,Host_NOC,Host_City,Sport,Event,NOC,Medal,Gold,Silver,Bronze,Home
0,2008,Afghanistan,37172386,520.90,CHN,Beijing,Taekwondo,Taekwondo Men's Flyweight,AFG,Bronze,False,False,True,False
1,2012,Afghanistan,37172386,520.90,GBR,London,Taekwondo,Taekwondo Men's Featherweight,AFG,Bronze,False,False,True,False
2,2000,Algeria,42228429,4114.72,AUS,Sydney,Boxing,Boxing Men's Light-Welterweight,ALG,Bronze,False,False,True,False
3,1996,Algeria,42228429,4114.72,USA,Atlanta,Boxing,Boxing Men's Middleweight,ALG,Bronze,False,False,True,False
4,2008,Algeria,42228429,4114.72,CHN,Beijing,Judo,Judo Men's Middleweight,ALG,Silver,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6267,2004,Zimbabwe,14439018,2147.00,GRE,Athina,Swimming,Swimming Women's 200 metres Individual Medley,ZIM,Bronze,False,False,True,False
6268,2008,Zimbabwe,14439018,2147.00,CHN,Beijing,Swimming,Swimming Women's 100 metres Backstroke,ZIM,Silver,False,True,False,False
6269,2008,Zimbabwe,14439018,2147.00,CHN,Beijing,Swimming,Swimming Women's 200 metres Backstroke,ZIM,Gold,True,False,False,False
6270,2008,Zimbabwe,14439018,2147.00,CHN,Beijing,Swimming,Swimming Women's 200 metres Individual Medley,ZIM,Silver,False,True,False,False


###### Grouping Data

In [ ]:
# starting here, we group our information by the year and country
# begin by getting the number of sports and events the country competed in
olympics = pd.DataFrame(games.groupby(['Year','NOC','Country'])[['Sport','Event']].nunique())
olympics

Sport  Event
Year NOC Country                    
1992 ALG Algeria            2      2
     ARG Argentina          1      1
     AUS Australia          9     26
     AUT Austria            2      2
     BAH Bahamas            1      1
...                       ...    ...
2016 UKR Ukraine            8     11
     USA United States     23    107
     UZB Uzbekistan         4     13
     VEN Venezuela          3      3
     VIE Vietnam            1      2

[548 rows x 2 columns]

In [ ]:
# next we include our population and GDP per capita
# these numbers are clearly not accurate for earlier years, but can still be used for 2020's predictions
df = pd.DataFrame(games.groupby(['Year','NOC','Country'])[['Population', 'GDP_Per_Capita']].mean())
df = df.groupby(['Year','NOC','Country']).mean()
olympics = olympics.merge(df, left_index=True, right_index=True)
olympics.head()

Sport  Event  Population  GDP_Per_Capita
Year NOC Country                                            
1992 ALG Algeria        2      2    42228429         4114.72
     ARG Argentina      1      1    44938712        11568.46
     AUS Australia      9     26    24982688        57395.92
     AUT Austria        2      2     8840521        51499.89
     BAH Bahamas        1      1      385640        32217.87

In [ ]:
# get the total number of medals won
df = pd.DataFrame(games.groupby(['Year','NOC','Country','Event'])[['Medal']].nunique())
df = df.groupby(['Year','NOC','Country']).sum()
olympics = olympics.merge(df, left_index=True, right_index=True)
olympics

Sport  Event  Population  GDP_Per_Capita  Medal
Year NOC Country                                                       
1992 ALG Algeria            2      2    42228429         4114.72      2
     ARG Argentina          1      1    44938712        11568.46      1
     AUS Australia          9     26    24982688        57395.92     27
     AUT Austria            2      2     8840521        51499.89      2
     BAH Bahamas            1      1      385640        32217.87      1
...                       ...    ...         ...             ...    ...
2016 UKR Ukraine            8     11    44622516         2931.98     11
     USA United States     23    107   324985539        68683.06    121
     UZB Uzbekistan         4     13    32955400         1532.37     13
     VEN Venezuela          3      3    28515829        16915.49      3
     VIE Vietnam            1      2    95540395         2566.60      2

[548 rows x 5 columns]

In [ ]:
# get the number of golds, silvers, and bronzes won
df = games.groupby(['Year','NOC','Country','Event'])[['Gold','Silver','Bronze']].sum()
df = df.clip(upper=1)
df = df.groupby(['Year','NOC','Country']).sum()
olympics = olympics.merge(df, left_index=True, right_index=True)
olympics.head()

Sport  Event  Population  ...  Gold  Silver  Bronze
Year NOC Country                              ...                      
1992 ALG Algeria        2      2    42228429  ...   1.0     0.0     1.0
     ARG Argentina      1      1    44938712  ...   0.0     0.0     1.0
     AUS Australia      9     26    24982688  ...   7.0     9.0    11.0
     AUT Austria        2      2     8840521  ...   0.0     2.0     0.0
     BAH Bahamas        1      1      385640  ...   0.0     0.0     1.0

[5 rows x 8 columns]

In [ ]:
# add a variable indicating home field
df = pd.DataFrame(games.groupby(['Year','NOC','Country'])[['Home']].mean())
olympics = olympics.merge(df, left_index=True, right_index=True)
olympics.head()

Sport  Event  Population  ...  Silver  Bronze   Home
Year NOC Country                              ...                       
1992 ALG Algeria        2      2    42228429  ...     0.0     1.0  False
     ARG Argentina      1      1    44938712  ...     0.0     1.0  False
     AUS Australia      9     26    24982688  ...     9.0    11.0  False
     AUT Austria        2      2     8840521  ...     2.0     0.0  False
     BAH Bahamas        1      1      385640  ...     0.0     1.0  False

[5 rows x 9 columns]

In [ ]:
# additional formatting
olympics = olympics.reset_index()
olympics['Gold']   = olympics['Gold'].astype('int64')
olympics['Silver'] = olympics['Silver'].astype('int64')
olympics['Bronze'] = olympics['Bronze'].astype('int64')
olympics['Home']   = olympics['Home'].astype('int64')

olympics = olympics.rename(columns={'Medal':'Medals', 'Sport':'Sports', 'Event':'Events'})
olympics = olympics[['Year', 'NOC', 'Country', 'Population', 'GDP_Per_Capita', 'Home',
                     'Sports', 'Events', 'Gold', 'Silver', 'Bronze', 'Medals']]
olympics.head()

,Year,NOC,Country,Population,GDP_Per_Capita,Home,Sports,Events,Gold,Silver,Bronze,Medals
0,1992,ALG,Algeria,42228429,4114.72,0,2,2,1,0,1,2
1,1992,ARG,Argentina,44938712,11568.46,0,1,1,0,0,1,1
2,1992,AUS,Australia,24982688,57395.92,0,9,26,7,9,11,27
3,1992,AUT,Austria,8840521,51499.89,0,2,2,0,2,0,2
4,1992,BAH,Bahamas,385640,32217.87,0,1,1,0,0,1,1


###### Saving to Google Drive

In [ ]:
# mount to drive
# from google.colab import drive
# drive.mount('/drive')

In [ ]:
# olympics.to_csv('/drive/My Drive/Colab Notebooks/olympics.csv');

###### Tables and Stats

In [ ]:
# NOCs with most medals since 1992
topNations = olympics.groupby(['NOC','Country'])[['Gold','Silver','Bronze','Medals']].sum()
topNations = topNations.sort_values(by='Medals', ascending=False).reset_index()
topNations.head(10)

,NOC,Country,Gold,Silver,Bronze,Medals
0,USA,United States,281,233,221,735
1,CHN,China,207,143,132,482
2,RUS,Russia,152,139,161,452
3,GER,Germany,123,111,145,379
4,AUS,Australia,78,101,107,286
5,GBR,United Kingdom,101,83,89,273
6,FRA,France,75,80,100,255
7,ITA,Italy,66,64,73,203
8,JPN,Japan,55,60,80,195
9,KOR,South Korea,71,63,59,193


In [ ]:
# performance of each host country since 1992
hosts = olympics[olympics['Home'] == 1].groupby(['Year','NOC','Country']).sum()
hosts = hosts.drop(['Population','GDP_Per_Capita','Home'], axis=1)
hosts = hosts.reset_index()
hosts

,Year,NOC,Country,Sports,Events,Gold,Silver,Bronze,Medals
0,1992,ESP,Spain,12,22,13,7,2,22
1,1996,USA,United States,21,89,44,32,25,101
2,2000,AUS,Australia,20,55,16,25,17,58
3,2004,GRE,Greece,10,16,6,6,4,16
4,2008,CHN,China,25,85,51,21,28,100
5,2012,GBR,United Kingdom,17,60,29,17,19,65
6,2016,BRA,Brazil,12,18,7,6,6,19


###### Adjusting Events for Tokyo Games

In [ ]:
# get mean of sports and events competed in for last 3 Olympics
# this will be the basis for these variables when predicting medals for the 2021 Games
previous_medals = olympics[olympics['Year'] >= 2008]
previous_medals = previous_medals.groupby(['NOC','Country'])[['Sports','Events']].mean().astype('int64')
previous_medals = previous_medals.reset_index()
previous_medals

,NOC,Country,Sports,Events
0,AFG,Afghanistan,1,1
1,ALG,Algeria,1,1
2,ARG,Argentina,4,4
3,ARM,Armenia,2,4
4,AUS,Australia,12,36
...,...,...,...,...
102,USA,United States,21,96
103,UZB,Uzbekistan,3,7
104,VEN,Venezuela,1,1
105,VIE,Vietnam,1,1


In [ ]:
# save to Drive
# previous_medals.to_csv('/drive/My Drive/Colab Notebooks/adjTokyo.csv');